## Dataframe of Louisiana Stations with Discharge Records

This notebook conducts ETL on USGS flow rate data in Louisiana Region 5.

In [235]:
import numpy as np
import requests
import datetime
import json
import os
import pandas as pd
from bs4 import BeautifulSoup
import geopandas as gpd
from shapely.geometry import Point
import zipfile

### Function to get List of discharge stations by state

In [236]:
def get_stations_by_date(state='la'):
    start_date='1995-01-01'

    today = datetime.datetime.today()
    today = datetime.datetime.strftime(today, '%Y-%m-%d')

    url = 'https://nwis.waterdata.usgs.gov/usa/nwis/uv'

    params = {'referred_module':'sw',
              'state_cd':state,
              'index_pmcode_00060':'1',
              'group_key':'NONE',
              'format':'sitefile_output',
              'sitefile_output_format':'html_table',
              'column_name': ['agency_cd', 'site_no', 'station_nm', 'site_tp_cd', 'dec_lat_va', 'dec_long_va'],
              'range_selection':'date_range',
              'begin_date':start_date,
              'end_date':today, 
              'date_format':'YYYY-MM-DD',
              'rdb_compression':'file',
              'list_of_search_criteria':'state_cd,realtime_parameter_selection'}

    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.content)
    stations = soup.find_all('table')[1]

    heads = [head.text for head in stations('th')]
    cells = [[cell.text for cell in row('td')] for row in stations('tr')]

    # row no. 0 is empty
    cells.pop(0)

    table = pd.DataFrame(cells, columns=heads)

    table['Dec. Lat.'] = table['Dec. Lat.'].astype('float32')
    table['Dec. Lon'] = table['Dec. Lon'].astype('float32')
    table = gpd.GeoDataFrame(table, geometry=[Point(xy) for xy in zip(table['Dec. Lon'],table['Dec. Lat.'])], crs='EPSG:4269')
    return table

### Download gauge information

In [237]:
stations = get_stations_by_date(state='la')
stations.head()

,Agency,Site Number,Site Name,Site type,Dec. Lat.,Dec. Lon,Cooraccr.,Dec.lat/longdatum,geometry
0,USGS,02489500,"Pearl River near Bogalusa, LA",Stream,30.793243,-89.820908,U,NAD83,POINT (-89.82091 30.79324)
1,USGS,02492000,"Bogue Chitto River near Bush, LA",Stream,30.629356,-89.897293,U,NAD83,POINT (-89.89729 30.62936)
2,USGS,02492111,"WILSON SLOUGH NEAR WALKIAH BLUFF, MS",Stream,30.570278,-89.811111,F,NAD83,POINT (-89.81111 30.57028)
3,USGS,07348700,"Bayou Dorcheat near Springhill, LA",Stream,32.994583,-93.396561,U,NAD83,POINT (-93.39656 32.99458)
4,USGS,07349450,"Bodcau Bayou near Springhill, LA",Stream,33.004025,-93.516838,U,NAD83,POINT (-93.51684 33.00402)


### Load Region 5 Polygons

In [238]:
regions = gpd.read_file('shp/R5_HUC8.shp')
regions.head()

,OBJECTID,TNMID,METASOURCE,SOURCEDATA,SOURCEORIG,SOURCEFEAT,LOADDATE,GNIS_ID,AREAACRES,AREASQKM,STATES,HUC8,NAME,Shape_Leng,Shape_Area,LWI_Region,geometry
0,1081,{76667D32-EA28-43B7-968F-7273107E9329},None,None,None,None,2016-06-06,0,1276035.66,5163.94,LA,08080101,Atchafalaya,5.483595,0.483664,5,"POLYGON ((-91.74461 31.14591, -91.74466 31.145..."
1,1082,{B6C83DDE-92C1-49D8-AB2B-4861E057BC15},None,None,None,None,2012-06-11,0,891219.80,3606.64,LA,08080201,Mermentau Headwaters,3.229451,0.338922,5,"POLYGON ((-92.62559 30.87683, -92.62488 30.876..."
2,2047,{16C4730F-338E-4146-986F-56E8DB2BD8A7},None,None,None,None,2012-06-11,0,1416137.17,5730.91,LA,08080102,Bayou Teche,7.253789,0.539654,5,"POLYGON ((-92.66793 31.39139, -92.66750 31.390..."
3,2048,{D464005C-5979-44DC-95A4-E2EC9AFA0A9A},None,None,None,None,2016-06-06,0,1289163.35,5217.06,LA,08080103,Vermilion,4.468562,0.487019,5,"POLYGON ((-92.08111 30.56625, -92.07998 30.566..."
4,2049,{290E1D06-3366-4E2B-A60A-7F7E534008BF},None,None,None,None,2016-06-06,0,1646620.00,6663.64,LA,08080202,Mermentau,4.715572,0.622428,5,"POLYGON ((-92.67131 30.40172, -92.67121 30.394..."


### Get HUC8 number and Basin for stations within Region 5 

In [239]:
def get_basin(point, regions):
    for basin, polygon in zip(regions['NAME'], regions['geometry']):
        if point.within(polygon):
            return basin

In [240]:
def get_huc(point, regions):
    for huc, polygon in zip(regions['HUC8'], regions['geometry']):
        if point.within(polygon):
            return huc

### Drop gauges outside Region 5

In [241]:
stations['Basin'] = stations['geometry'].apply(lambda x: get_basin(point=x, regions=regions))
stations.dropna(subset=['Basin'], inplace=True)

stations['Huc8'] = stations['geometry'].apply(lambda x: get_huc(point=x, regions=regions))
stations.head()

,Agency,Site Number,Site Name,Site type,Dec. Lat.,Dec. Lon,Cooraccr.,Dec.lat/longdatum,geometry,Basin,Huc8
49,USGS,07381490,"Atchafalaya River at Simmesport, LA",Stream,30.982500,-91.798332,S,NAD83,POINT (-91.79833 30.98250),Atchafalaya,08080101
50,USGS,07381590,"Wax Lake Outlet at Calumet, LA",Stream,29.697987,-91.372887,S,NAD83,POINT (-91.37289 29.69799),Atchafalaya,08080101
51,USGS,07381600,"Lower Atchafalaya River at Morgan City, LA",Stream,29.692820,-91.211937,S,NAD83,POINT (-91.21194 29.69282),Atchafalaya,08080101
58,USGS,073816537,"Castille Pass near Morgan City, LA",Estuary,29.419445,-91.276947,S,NAD83,POINT (-91.27695 29.41945),Atchafalaya,08080101
59,USGS,07381670,"GIWW at Bayou Sale Ridge near Franklin, LA",Stream,29.680834,-91.470558,S,NAD83,POINT (-91.47056 29.68083),Atchafalaya,08080101


### Download raw data in JSON and TXT formats

In [242]:
for basin, huc in zip(regions['NAME'], regions['HUC8']):
    basin = basin.replace(' ', '_')
    if not os.path.isdir(f'raw/{huc}_{basin}'):
        try:
            os.mkdir('raw')
            os.mkdir(f'raw/{huc}_{basin}')
        except FileExistsError:
            os.mkdir(f'raw/{huc}_{basin}')

In [243]:
import json

def download_data(gauge, huc, basin):
    basin = basin.replace(' ', '_')
    
    if os.path.exists(f'raw/{huc}_{basin}/{gauge}.zip') and os.path.exists(f'raw/{huc}_{basin}/{gauge}.txt'):
        return None

    url = 'https://waterservices.usgs.gov/nwis/iv/'

    start_date='1995-01-01'

    today = datetime.datetime.today()
    today = datetime.datetime.strftime(today, '%Y-%m-%d')

    params = {'format':'json',
              'sites':gauge,
              'startDT':start_date,
              'endDT':today,
              'parameterCd':'00060',
              'siteStatus':'all'}

    response = requests.get(url, params=params)
    data = json.loads(response.content)
    
    with zipfile.ZipFile(f'raw/{huc}_{basin}/{gauge}.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        zf.writestr(f'{gauge}.json', json.dumps(data, indent=4))
        zf.close()
    #get tab delimited txt    
    params['format'] = 'rdb'
    response = requests.get(url, params=params)
    with open(f'raw/{huc}_{basin}/{gauge}.txt', 'w') as outfile:
        outfile.write(response.content.decode())

In [244]:
for gauge, huc, basin in zip(stations['Site Number'], stations['Huc8'], stations['Basin']):
    download_data(gauge, huc, basin)

### Process and Plot Data

In [245]:
for basin, huc in zip(regions['NAME'], regions['HUC8']):
    basin = basin.replace(' ', '_')
    if not os.path.isdir(f'processed/{huc}_{basin}'):
        try:
            os.mkdir('processed')
            os.mkdir(f'processed/{huc}_{basin}')
        except FileExistsError:
            os.mkdir(f'processed/{huc}_{basin}')
    if not os.path.isdir(f'plots/{huc}_{basin}'):
        try:
            os.mkdir('plots')
            os.mkdir(f'plots/{huc}_{basin}')
        except FileExistsError:
            os.mkdir(f'plots/{huc}_{basin}')

In [246]:
from bokeh.plotting import figure, output_file, show, save
from bokeh.models.tools import HoverTool
from bokeh.models import NumeralTickFormatter

def plot(proc_df, number, name):

    TOOLS="xwheel_zoom,ywheel_zoom,crosshair,pan,reset,save"
    title = f'{number} {name}'
    p1 = figure(x_axis_type="datetime", title=title,
                tools=TOOLS, aspect_ratio=3, plot_width=1400,
                active_scroll='xwheel_zoom')

    p1.grid.grid_line_alpha=0.3
    p1.xaxis.axis_label = 'Date and Time (UTC)'
    p1.yaxis.axis_label = 'Discharge (cfs)'
    approved = proc_df[proc_df['Approved/Provisional']=='A']
    provisional = proc_df[proc_df['Approved/Provisional']=='P']
    p1.line(x='Date Time (UTC)', y='Q (cfs)', source=approved ,color='#1f77b4', legend_label='Approved Data')
    if provisional.shape[0]:
        p1.line(x='Date Time (UTC)', y='Q (cfs)', source=provisional ,color='#d62728', legend_label='Provisional Data')
    p1.left[0].formatter.use_scientific = False 
    p1.yaxis[0].formatter = NumeralTickFormatter(format="0,0")
    hover = HoverTool(tooltips=[('DateTime', '@{Date Time (UTC)}{%Y-%m-%d %H:%M}'),
                                ('Discharge', '@{Q (cfs)}{0,0}')],
              formatters={'@{Date Time (UTC)}': 'datetime'})

    p1.add_tools(hover)
    return p1

In [257]:
for idx in stations.index:
    basin = stations.loc[idx, 'Basin'].replace(' ', '_')
    huc = stations.loc[idx, 'Huc8']
    number = stations.loc[idx,'Site Number']
    path = f'raw/{huc}_{basin}/{number}.zip'
#     if os.path.exists(f'plots/{huc}_{basin}/{number}.html'):
#         continue
    with zipfile.ZipFile(path, 'r') as zf:
        data = json.load(zf.open(f'{number}.json'))
    noDataValue = data['value']['timeSeries'][0]['variable']['noDataValue']
    name = data['value']['timeSeries'][0]['sourceInfo']['siteName']
    number = data['value']['timeSeries'][0]['sourceInfo']['siteCode'][0]['value']
    raw = pd.DataFrame(data['value']['timeSeries'][0]['values'][0]['value'])
    raw = raw[['dateTime', 'value', 'qualifiers']]
    raw['dateTime'] = raw['dateTime'].apply(lambda x:pd.Timestamp(x))
    raw['value'] = raw['value'].astype('float32')
    raw.replace({noDataValue:np.nan}, inplace=True)
    raw_path = path.replace('.zip', '.csv')
    raw.to_csv(raw_path, index=False)
    adjusted = raw.copy()
    #convert time to UTC
    adjusted['dateTime'] = adjusted['dateTime'].apply(lambda x: x.tz_convert(0))
    adjusted['qualifiers'] = adjusted['qualifiers'].apply(lambda x:x[0])
    interval = adjusted['dateTime'].iloc[1] - adjusted['dateTime'].iloc[0]
    min_date = adjusted['dateTime'].iloc[0]
    max_date = adjusted['dateTime'].iloc[-1]
    index = pd.date_range(min_date, max_date, freq=interval)
    processed = pd.DataFrame(data=index, columns=['dateTime'])
    processed = pd.merge(processed, adjusted, left_on='dateTime', right_on='dateTime', how='outer')
    processed.rename(columns={'dateTime':'Date Time (UTC)',
                              'value':'Q (cfs)', 
                              'qualifiers':'Approved/Provisional'}, inplace=True)
    

    
    processsed_path = raw_path.replace('raw/','processed/')
    processed.sort_values('Date Time (UTC)', inplace=True)
    processed.to_csv(processsed_path, index=False)
    
    processed['Approved/Provisional'] = processed['Approved/Provisional'].fillna(method='ffill')
    assert processed['Approved/Provisional'].value_counts().shape[0] <= 2
    p = plot(processed, number, name)
    output_file(f'plots/{huc}_{basin}/{number}.html', mode='inline', title=f'{number} {name}')
    save(p)
    stations.loc[idx, 'min_Q'] = processed['Q (cfs)'].min()
    stations.loc[idx, 'min_date'] = processed.set_index('Date Time (UTC)').loc[:,'Q (cfs)'].idxmin()
    stations.loc[idx, 'max_Q'] = processed['Q (cfs)'].max()
    stations.loc[idx, 'max_date'] = processed.set_index('Date Time (UTC)').loc[:,'Q (cfs)'].idxmax()
    raw_url = f'https://raw.githubusercontent.com/sinaamini11/Louisiana_Region5_Discharges/main/raw/{huc}_{basin}/{number}.txt'
    stations.loc[idx, 'raw_url'] = raw_url
    proc_url = f'https://raw.githubusercontent.com/sinaamini11/Louisiana_Region5_Discharges/main/processed/{huc}_{basin}/{number}.csv'
    stations.loc[idx, 'processed_url'] = proc_url
    plot_url = f'https://github.com/sinaamini11/Louisiana_Region5_Discharges/blob/main/plots/{huc}_{basin}/{number}.html'
    stations.loc[idx, 'plot_url'] = 'https://htmlpreview.github.io/?'+plot_url

### Folium Map

In [258]:
map_data = stations.copy()
map_data.set_index('Site Number', inplace=True)

In [267]:
import folium


m = folium.Map(location=[29.847986, -91.608168],
                zoom_start=8)

for station in map_data.index:
    lat = map_data.loc[station, 'Site Name']
    lat = map_data.loc[station, 'Dec. Lat.']
    long = map_data.loc[station, 'Dec. Lon']
    max_Q = map_data.loc[station, 'max_Q']
    max_date = map_data.loc[station, 'max_date']
    min_Q = map_data.loc[station, 'min_Q']
    min_date = map_data.loc[station, 'min_date']
    raw_url = map_data.loc[station, 'raw_url']
    processed_url = map_data.loc[station, 'processed_url']
    plot_url = map_data.loc[station, 'plot_url']
    basin = map_data.loc[station, 'Basin']
    with open('popup_format.html','r') as f:
        text = f.read()
    text = text.replace('{station_id}', f'{station} {name}')
    text = text.replace('{lattitude}', str(round(lat,6)))
    text = text.replace('{longitude}', str(round(long,6)))
    text = text.replace('{max_Q}', f'{max_Q:,.1f}' + ' cfs')
    text = text.replace('{max_date}', str(max_date)[:-6])
    text = text.replace('{min_Q}', f'{min_Q:,.1f}' + ' cfs')
    text = text.replace('{min_date}', str(min_date)[:-6])
    text = text.replace('raw_url', raw_url)
    text = text.replace('processed_url', processed_url)
    text = text.replace('plot_url', plot_url)
    text = text.replace('{Basin}', basin)
    page = folium.Html(text, script=True)
    popup = folium.Popup(page, max_width=400,min_width=400)
    folium.Marker([lat, long], icon=folium.Icon(icon='tint', prefix='fa', color='red'),
              popup = popup, tooltip=f'{station} {name}').add_to(m)        

m.save('./docs/index.html')

In [268]:
m


In [266]:
f'{max_Q:,.1f}'

'8,280.0'